In [1]:
import os
import time
import motor.motor_asyncio
import poe_lib
import tqdm
import re

from poe_lib.objects import Note

In [2]:
client = motor.motor_asyncio.AsyncIOMotorClient(os.environ['MONGO_URL'])

In [3]:
coll = client.trade.items

In [5]:
map_keys = list(poe_lib.objects.Note.CURRENCY_REMAP.keys())
map_keys

['alch',
 'alt',
 'armour',
 'awakened-sextant',
 'bauble',
 'blessed',
 'cart',
 'chanc',
 'chance',
 'chaos',
 'Chaos',
 'CHAOS',
 'chisel',
 'chrom',
 'chrome',
 'div',
 'divine',
 'ex',
 'exa',
 'exalt',
 'exalted orb',
 'exalted-orb',
 'exalted',
 'Exalted',
 'fusing',
 'gcp',
 'jewellers',
 'kalandra',
 'mir',
 'mirror',
 'regal',
 'regret',
 'scour',
 'silver',
 'vaal',
 'whetstone']

In [4]:
my_filter = {'_price.unit': { '$in': map_keys}}

NameError: name 'map_keys' is not defined

In [ ]:
pipeline = [
    {
        '$match': my_filter
    },
    { 
        '$lookup': {
            'from': 'stashes',
            'localField': '_stash_id',
            'foreignField': 'id',
            'as': 'stash'
        }
    },
    {
        '$unwind': {
            'path': '$stash'
        }
    },
    {
        '$project': {
                'note': 1,
                '_price': 1,
                'stash.stash': 1,
                '_stash_id': 1,
        }
    }
]

total_items = await coll.count_documents(my_filter)
print(f'{total_items:,}')


with tqdm.tqdm(total=total_items, unit='item', smoothing=0.001) as pbar:
    async for item in coll.aggregate(pipeline):
        pbar.update(1)
        item_note = poe_lib.objects.Note(item.get('note', None))
        if not item_note.is_valid:
            item_note = poe_lib.objects.Note(item['stash']['stash'])
            if not item_note.is_valid:
                 continue
        item['_price'] = {
            'type': item_note.type,
            'unit': item_note.unit,
            'value': item_note.value,
        }
        await coll.find_one_and_update({'_id': item['_id']}, {'$set': {**item}})

In [ ]:
types = await coll.distinct('_price.unit', {'extended.category': 'currency'})
with open('units.txt', 'w') as fp:
    for t in types:
        fp.write(t+'\n')

In [ ]:
types = await coll.distinct('typeLine', {'extended.category': 'currency'})
with open('typeLines.txt', 'w') as fp:
    for t in types:
        fp.write(t+'\n')

In [6]:
from collections import defaultdict

my_filter = {'_price.unit': { '$exists': True}}

types = defaultdict(lambda: 0)
async for t in coll.find(my_filter, projection={'_price.unit': 1}):
    types[t['_price']['unit']] += 1
[x for x in sorted(types.items(), key=lambda x: x[1], reverse=True) if x[0] not in list(poe_lib.objects.Note.CURRENCY_REMAP.values())]

[('aug', 671),
 ('master-sextant', 621),
 ('engineers', 563),
 ('fuse', 459),
 ('chaos-', 432),
 ('c', 381),
 ('transmute', 380),
 ('veiled-chaos-orb', 315),
 ('facetors', 281),
 ('simple-sextant', 255),
 ('annul', 250),
 ('chaosquad', 191),
 ('chaosc', 181),
 ('offer', 178),
 ('chaosQuad', 168),
 ('scrap', 158),
 ('chaos-unq', 156),
 ('chaos-LENG', 147),
 ('chaosBLUE', 144),
 ('prime-sextant', 137),
 ('chaosGREEN', 134),
 ('chaosRED', 129),
 ('chaoss', 122),
 ('rogues-marker', 121),
 ('stacked-deck', 111),
 ('chaos-jewel', 110),
 ('exalted-shard', 110),
 ('chaosfff', 108),
 ('jew', 105),
 ('a', 98),
 ('chaos-THE', 95),
 ('ch', 90),
 ('ancient-orb', 87),
 ('wisdom', 82),
 ('chaosInfluenced', 81),
 ('portal', 81),
 ('chaosx', 76),
 ('chaosss', 60),
 ('awakeners-orb', 57),
 ('chao', 55),
 ('chaos---', 55),
 ('simulacrum', 55),
 ('p', 54),
 ('bound-fossil', 53),
 ('offer-gift', 53),
 ('hunters-exalted-orb', 52),
 ('fertile-catalyst', 51),
 ('lookup', 50),
 ('chaos-tab', 49),
 ('orb-of-unm

In [8]:

[x for x in sorted(types.items(), key=lambda x: x[1], reverse=True) if (x[0] not in list(poe_lib.objects.Note.CURRENCY_REMAP.values())) and x[1] > 100]

[('aug', 671),
 ('master-sextant', 621),
 ('engineers', 563),
 ('fuse', 459),
 ('chaos-', 432),
 ('c', 381),
 ('transmute', 380),
 ('veiled-chaos-orb', 315),
 ('facetors', 281),
 ('simple-sextant', 255),
 ('annul', 250),
 ('chaosquad', 191),
 ('chaosc', 181),
 ('offer', 178),
 ('chaosQuad', 168),
 ('scrap', 158),
 ('chaos-unq', 156),
 ('chaos-LENG', 147),
 ('chaosBLUE', 144),
 ('prime-sextant', 137),
 ('chaosGREEN', 134),
 ('chaosRED', 129),
 ('chaoss', 122),
 ('rogues-marker', 121),
 ('stacked-deck', 111),
 ('chaos-jewel', 110),
 ('exalted-shard', 110),
 ('chaosfff', 108),
 ('jew', 105)]